<a href="https://colab.research.google.com/github/Gideon1712/SimpleTrading/blob/main/SimpleTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install Flask

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session

app = Flask(SimpleTrading)
app.secret_key = 'a_secure_secret_key'